In [1]:
pip install dgl-cu111 -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
cpu_tensor = torch.Tensor([0,1])

# https://pytorch.org/docs/stable/tensor_attributes.html#torch.torch.device
# 어느 장치(cpu 혹은 gpu)에 텐서를 올릴지 지정합니다.
# 아래는 torch.device라는 함수를 사용해 gpu로 장치를 지정합니다. 
device = torch.device('cuda')

# https://pytorch.org/docs/stable/cuda.html?highlight=available#torch.cuda.is_available
# gpu가 사용 가능한지 확인해줍니다.
if torch.cuda.is_available():
  
  # https://pytorch.org/docs/stable/tensors.html?highlight=#torch.Tensor.to
  # cpu에 있었던 텐서를 to 함수를 이용해 지정해놓은 장치(여기서는 gpu)로 올려줍니다.
  gpu_tensor = cpu_tensor.to(device)
  print(gpu_tensor)


# to(device) 붙이면 gpu 연산 가능

tensor([0., 1.], device='cuda:0')


In [4]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

In [5]:
# data load

import dgl
from dgl.data import FB15kDataset
import torch

dataset = FB15kDataset()
graph = dataset[0]

# edata = returns a dict[str, Tensor]

train_mask = graph.edata['train_mask']
val_mask = graph.edata['val_mask']
test_mask = graph.edata['test_mask']

head, tail = graph.edges('uv')
rel = graph.edata['etype']
triplet = torch.stack([head, rel, tail], dim=1)

train = triplet[train_mask]
val = triplet[val_mask]
test = triplet[test_mask]

Using backend: pytorch


# entities: 14951
# relations: 1345
# training edges: 483142
# validation edges: 50000
# testing edges: 59071
Done loading data from cached files.


Model

In [6]:
class TransE(nn.Module):

  def __init__(self,num_entity, num_rel ,embed_dim, margin):
    super(TransE, self).__init__()

    bound = 6 / (embed_dim ** 0.5)

    self.embed_dim = embed_dim
    self.margin = margin

    # entity initialize
    self.entity_embed = nn.Embedding(num_entity, embed_dim)
    self.entity_embed.weight.data = torch.nn.init.uniform_(self.entity_embed.weight.data, a= - bound, b= bound)
    self.entity_embed.weight.data = F.normalize(self.entity_embed.weight.data,p=2.0, dim=1, eps=1e-12)

    # relation initialize
    self.rel_embed = nn.Embedding(num_rel, embed_dim)
    self.rel_embed.weight.data = torch.nn.init.uniform_(self.rel_embed.weight.data, a= - bound, b= bound)
    self.rel_embed.weight.data = F.normalize(self.rel_embed.weight.data,p=2.0, dim=1, eps=1e-12)
    

  def forward(self, batch, c_batch): # c_batch = corrupted batch
    batch = batch.to(device)
    c_batch = c_batch.to(device)
    
    head, rel, tail = batch[0], batch[1], batch[2]
    c_head, c_tail = c_batch[0], c_batch[2] # c_ = corrupted

    # positive sample
    h = self.entity_embed(head).to(device)
    r = self.rel_embed(rel).to(device)
    t = self.entity_embed(tail).to(device)

    # negative (= corrupted) sample
    c_h = self.entity_embed(c_head).to(device)
    c_t = self.entity_embed(c_tail).to(device)

    # L1_dist: np.abs(real-pred) / L2_dist = (real-pred)**2
    # use L1_dist
    pos_dist = torch.norm(h+r - t, 2, dim = 0).to(device)
    neg_dist = torch.norm(c_h+r - c_t, 2, dim = 0).to(device)
    
    return pos_dist, neg_dist

main

In [ ]:
import time

# sampling corrupt function

def sampling_corrupt(sample):
  
  temp_sample = sample.clone().detach() # not change original sample data

  if random.random() > 0.5:

    while(1):
      
      corrupt_head = random.choice(head)
      if sample[0] != corrupt_head:
        temp_sample[0] = corrupt_head
        return temp_sample
        break
      
  else:
    while(1):
      corrupt_tail = random.choice(tail)
      if sample[2] != corrupt_tail:
        temp_sample[2] = corrupt_tail
        return temp_sample
        break

entity = graph.nodes()

model = TransE(num_entity = len(entity), num_rel = len(rel), embed_dim = 50, margin = 0.5)
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01)

# for mini-batch sampling
from torch.utils.data import DataLoader
batch_size = 4800

s_batch = DataLoader(
    train,
    batch_size = batch_size,
    shuffle=True
)

epochs = 10

for epoch in range(epochs + 1):

  for batch_idx, s in enumerate(s_batch):
      start = time.time()
      s.to(device)

      # sample corrupted triplets
      corrupeted_sample = 0
      corrupted_batch = []
     
      for i in range(len(s)):
        corrupted_sample = sampling_corrupt(s[i])
        corrupted_batch.append(corrupted_sample)

      # reshape for triplet form
      corrupted_batch = torch.stack(corrupted_batch)
      corrupted_batch = corrupted_batch.reshape(-1,3)
      corrupted_batch.to(device)
      
      pos_dist_group = []
      neg_dist_group = []
      optimizer.zero_grad()

      for i in range(len(s)):
        pos_dist, neg_dist = model(s[i], corrupted_batch[i])
        pos_dist_group.append(pos_dist)
        neg_dist_group.append(neg_dist)

      pos_dist_group = torch.stack(pos_dist_group)
      neg_dist_group = torch.stack(neg_dist_group)

      loss_func = nn.MarginRankingLoss(margin = 0.5) # margin rank = max(0,−y∗(x1−x2)+margin)
      loss = loss_func(pos_dist_group, neg_dist_group,torch.ones_like(pos_dist_group)*-1)
      loss.backward()
      optimizer.step()


      end = time.time()
      collapsed = end - start
      print(f"collapsed time: {collapsed}")
      print(f"Epoch {epoch}/{epochs}, Batch {batch_idx}/{len(s_batch)}, Loss: {loss}")
  

  print("\n")

collapsed time: 21.128891229629517
Epoch 0/10, Batch 0/202, Loss: 0.5000765323638916
collapsed time: 21.123464107513428
Epoch 0/10, Batch 1/202, Loss: 0.4978112280368805
collapsed time: 21.030547618865967
Epoch 0/10, Batch 2/202, Loss: 0.49844759702682495
collapsed time: 21.08294987678528
Epoch 0/10, Batch 3/202, Loss: 0.5015605688095093
collapsed time: 21.09828472137451
Epoch 0/10, Batch 4/202, Loss: 0.49757757782936096
collapsed time: 21.020882606506348
Epoch 0/10, Batch 5/202, Loss: 0.5013521909713745
collapsed time: 21.09777855873108
Epoch 0/10, Batch 6/202, Loss: 0.5002166032791138
collapsed time: 21.107596158981323
Epoch 0/10, Batch 7/202, Loss: 0.5007187724113464
collapsed time: 21.061738967895508
Epoch 0/10, Batch 8/202, Loss: 0.5005778670310974
collapsed time: 21.098220109939575
Epoch 0/10, Batch 9/202, Loss: 0.5003821849822998
collapsed time: 21.056906700134277
Epoch 0/10, Batch 10/202, Loss: 0.5039043426513672
collapsed time: 21.02669334411621
Epoch 0/10, Batch 11/202, Loss:

In [ ]:
a = []
len(a)

In [ ]:
output = []
# input1 = torch.Tensor([[1,2,3],[7,8,9]])
# input2 = torch.Tensor([[4,5,6],[10,11,12]])

input1 = torch.Tensor([1,2,3])
input2 = torch.Tensor([[4,5,6]])

print(input1)

input1 = [np.array(input1)]

print(input1)

input1 = torch.Tensor(input1)
print(input1)
output.append(input1)
output.append(input2)

print(output)

result = torch.cat(output, dim=0)
print(result)

# for ch in range(in_channels):
#     tensor = my_fn(input_batch[:,ch,:,:])   #transform from (64, 1, 224, 224) to (64, 32, 224, 224)
#     outputs.append(tensor)
# result = torch.cat(outputs, dim=1)  #shape (64, 32*in_channels, 224, 224

In [ ]:
a = [torch.Tensor([0]),torch.Tensor([1]),torch.Tensor([3])]
b = torch.stack(a)
print(a)
print(b)

In [ ]:
torch.numel(torch.Tensor([0]))

In [ ]:
# for corrupt sample
k = 0

for batch_idx, s in enumerate(s_batch):
  
    corrupted = torch.Tensor([0])
    # sample corrupted triplets
    corrupt_sample = 0
    print(f"우씨: {s}")

    for i in range(len(s)):
      corrupt_sample = sampling_corrupt(s[i])
      if torch.numel(corrupted) == 1: #corrupted = tensor([0])
        corrupted = corrupt_sample
      else:
        corrupted = torch.cat((corrupted, corrupt_sample))
      corrupted = corrupted.reshape(-1,3)
      print(f"아으: {corrupted}")
    
    print(f"완성: {corrupted}")

    k += 1 # for test
    if k > 5:
      break

In [ ]:
# for corrupt sample
k = 0

for batch_idx, s in enumerate(s_batch):
  
    temp_list = []
    corrupted = torch.Tensor([0])
    # sample corrupted triplets
    corrupt_sample = 0
    # print(f"우씨: {s}")

    for i in range(len(s)):
      corrupt_sample = sampling_corrupt(s[i])
      corrupted = corrupt_sample
      temp_list.append(corrupted)
      

      # print(f"아으: {corrupted}")
    
    # print(f"완성: {corrupted}")
    print(temp_list)
    output = torch.cat(temp_list, dim = 0)
    print(output)
    k += 1 # for test
    if k > 5:
      break

In [ ]:
train

In [ ]:
import torch
a = torch.arange(10).float()
b = torch.arange(1, 11).float()

In [ ]:
a.requires_grad = True
b.requires_grad = True

In [ ]:
# # embedding initialize & normalization function

# def uniform_initialize(val, bound):
#       val = torch.nn.init.uniform_(torch.empty(val.shape), a= - bound, b= bound)
#       return val

# def normalization(val):
#   temp_val = torch.Tensor([0])
#   for v in val:
#     norm_v = torch.Tensor([torch.norm(v)])
#     temp_val = torch.cat((temp_val,norm_v))

#   val = temp_val[1:] # 초기화 한 값 제거
#   return val

In [ ]:
# sampling corrupt function

def sampling_corrupt(sample):
  
  temp_sample = sample.clone().detach() # not change original sample data

  if random.random() > 0.5:

    while(1):
      
      corrupt_head = random.choice(head)
      if sample[0] != corrupt_head:
        temp_sample[0] = corrupt_head
        return temp_sample
        break
      
  else:
    while(1):
      corrupt_tail = random.choice(tail)
      if sample[2] != corrupt_tail:
        temp_sample[2] = corrupt_tail
        return temp_sample
        break

In [ ]:
# # 시간이 너무 오래걸림, 모르는 함수가 있는건가 ?!

# # relation set uniform으로 초기화 및 정규화, entity 초기화

# k = 50 # embedding dimension

# bound = 6/np.sqrt(k)

# entity = graph.nodes()

# # initialization rel, entity

# rel_norm = uniform_initialize(rel, bound)
# entity_norm = uniform_initialize(entity, bound)

# # relation & entity normalization

# rel_norm = normalization(rel_norm)
# entity_norm = normalization(entity_norm)

In [ ]:
def margin_loss(pos_dist, neg_dist):
  
  loss_func = nn.MarginRankingLoss()
  cost = loss_func(pos_dist,neg_dist)

  return cost


In [ ]:
def dist_L1(real,pred):
  return np.abs(real-pred)

In [ ]:
def dist_L2(real,pred):
  return (real-pred)**2

In [ ]:
next(iter(s_batch))

In [ ]:
# initiallization for new set of batch = T_batch

epochs = 10
t_batch = 0
pos_dist = 0
neg_dist = 0
k = 0

# loop, sample(h, l, t)에 대한 for문

optimizer = optim.SGD([rel_norm,entity_norm], lr=0.01, momentum=0.9)

for epoch in range(epochs + 1):

  for batch_idx, s in enumerate(s_batch):

    # sample corrupted triplets
    s_corrupt = torch.Tensor([0])
    corrupt_sample = 0

    for i in range(len(s)):

      corrupt_sample = sampling_corrupt(s[i])

      temp_pos_dist = torch.Tensor([dist_L1(s[i][0]+ s[i][1], s[i][2])])
      temp_neg_dist = torch.Tensor([dist_L1(corrupt_sample[0]+ corrupt_sample[1], corrupt_sample[2])])

      if t_batch == 0:
        pos_dist = temp_pos_dist
        neg_dist = temp_neg_dist

      else:
        pos_dist = torch.cat((pos_dist,temp_pos_dist))
        print(pos_dist)
        neg_dist = torch.cat((neg_dist,temp_neg_dist))
        
      t_batch += 1

    for p in range(len(pos_dist)):
      # optimizer.zero_grad()
      cost = margin_loss(pos_dist[p], neg_dist[p])
      cost.backward()
      optimizer.step()

    print(f"Epoch {epoch}/{epochs}, Batch {batch_idx}/{batch_num}, Cost: {cost}")


    k += 1
    if k > 6:
      break 


In [ ]:
from torch.utils.data import DataLoader # for mini-batch sampling

# loop 시작

epochs = 10
batch_num  = 100 # batch size
s_batch = 0

for epoch in range(epochs + 1):

  # sample (h,l,t) mini batch for mini-batch - SGD = S_batch

  s_batch = DataLoader(
      train,
      batch_size = batch_num,
      shuffle=True
  )

  # initiallization for new set of batch = T_batch
  t_batch = 0
  # cat positive & negative sample
  mix_samples(s_batch)
    
# end


In [ ]:
# update cost function
    
pos_dist = dist_L1()
neg_dist = dist_L1()

cost = margin_loss(pos_dist, neg_dist)

print(f"Epoch {epoch}/{epochs}, Batch {batch_idx}/{batch_num}, Cost: {cost}")

최종 버전

In [ ]:
a = nn.Embedding(5,3)
a.weight

In [ ]:
a.weight.data

In [ ]:
a(torch.LongTensor([1]))

In [ ]:
torch.nn.init.uniform_(a.weight.data, a= - 1, b= 1)

In [ ]:
# class Triplets:
#   def __init__(self, sample):
#     self.head = sample[0]
#     self.rel = sample[1]
#     self.tail = sample[2]